In [1]:
import praw
import pandas as pd
import time

# Initializing the Reddit API client with my unique credentials
reddit = praw.Reddit(client_id='9iNmvNiFvQs_V_cu24vphQ',
                     client_secret='wxmYcvLVW7E1dxZJ1HN1VLFkXlTb0A', 
                     password='Reddit!5',
                     user_agent='Jacquelyn', 
                     username='capstoneundercover')

# Specifing the desired subreddit and retrieving the posts
ai_subreddit = reddit.subreddit('Autoimmune')
post_list = []
for post in ai_subreddit.new(limit=500):  # Retriveing the 500 newest posts at this given time
    post_list.append([post.title, post.selftext, post.score, post.num_comments, post.url])
    if len(post_list) >= 500:  # Stops after retrieving 500 posts to ensure that my data stays accurate and uniform throughout all locations
        break
    time.sleep(2)  # Using a 2-second delay between requests to ensure I don't go over Reddit's API rates
#this was taken on: march 13th, 2023
# Extracting post titles, bodies, scores (upvotes), number of comments, and url
post_titles = []
post_bodies = []
post_scores = []
post_comments = []
post_url = []
for post in post_list:
    post_titles.append(post[0])
    post_bodies.append(post[1])
    post_scores.append(post[2])
    post_comments.append(post[3])
    post_url.append(post[4])

# Creating a DataFrame
df = pd.DataFrame({'post_title': post_titles, 'post_body': post_bodies, 'post_score': post_scores,
                  'post_comments': post_comments, 'post_url': post_url})

# Saving the DataFrame to a CSV file
df.to_csv('ai_posts.csv', index=False)

In [2]:
# Ensuring that the previous code has successfully transferred to my CSV file
# This is not required, but I like the reassurance 
try:
    df.to_csv('ai_posts.csv', index=False)
    print("CSV file successfully saved.")
except Exception as e:
    print("Error occurred while saving CSV:", e)

CSV file successfully saved.


In [8]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if isinstance(text, str):  # Checking to see if the input is a string
        tokens = word_tokenize(text)  
        tokens = [word.lower() for word in tokens if word.isalpha()]  # Removing punctuation and converting to lowercase
        custom_stop_words = ['got', 'sees', 'today', 'told', 'much', 'we', 'think',
                            'called', 'since']  # Add your custom stop words here
        stop_words.update(custom_stop_words)  # Update the stop_words set with custom stop words
        tokens = [word for word in tokens if word not in stop_words]  # Removing stop words
        return " ".join(tokens)
    else:
        return ""  # Return an empty string for NaN values

# Assuming 'data' is a DataFrame that has been defined earlier
data = df.dropna(subset=['post_body'])  # Removing rows with NaN values in 'post_body' column
data['clean_text'] = data['post_body'].apply(preprocess_text)

data.to_csv('cleaned_ai_data.csv', index=False)

In [11]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('cleaned_ai_data.csv') # Now you can use the 'df' DataFrame in your code

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Create an instance of the sentiment analyzer
sentimentAnalyzer = SentimentIntensityAnalyzer()
for post in df.index:
    if isinstance(df['clean_text'][post], str):
        print(df['clean_text'][post][0:100])
        print(sentimentAnalyzer.polarity_scores(df['clean_text'][post]))

first time posting group open thought comments advice history endometriosis total hysterectomy age c
{'neg': 0.221, 'neu': 0.721, 'pos': 0.058, 'compound': -0.9785}
result blood work borderline positive ana lab considered positive result pattern speckled time norma
{'neg': 0.199, 'neu': 0.582, 'pos': 0.218, 'compound': 0.5106}
included ordered ena panel
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
first rheuma brushed finally saw second one rheuma family friend suffering similar symptoms extremel
{'neg': 0.158, 'neu': 0.65, 'pos': 0.193, 'compound': 0.6344}
okay lupus years old im couple months ago biopsy trying diagnose biopsy results could differentiate 
{'neg': 0.125, 'neu': 0.768, 'pos': 0.107, 'compound': -0.4019}
looking decent bilirubin frequent appearance symptoms hit peak leukocytes new colored urine new prev
{'neg': 0.0, 'neu': 0.916, 'pos': 0.084, 'compound': 0.2732}
hi hope well feel really silly writing aware lot people probably suffer feel scared lonely need help

In [13]:
from textblob import TextBlob

# Replace NaN values in 'clean_text' column with an empty string
df['clean_text'] = df['clean_text'].fillna('')

# Add a new column to the DataFrame to store the sentiment scores
df['sentiment_scores'] = df['clean_text'].apply(lambda text: TextBlob(str(text)).sentiment.polarity)

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_ai_data_with_sentiment.csv', index=False)

In [15]:
import pandas as pd

# Loading the CSV file into the DataFrame again
df = pd.read_csv('updated_ai_data_with_sentiment.csv')

# Exporting the DataFrame to an Excel file located on my computer for easier viewing
df.to_excel('final_ai_posts.xlsx', index=False)